# Data Processing Pipeline

This notebook implements a 10-stage pipeline for processing multi-omics cancer data:

1. **Setup**: Clone/update repository
2. **Data Collection**: Create tumor-type TSVs from Kaggle datasets
3. **Column Alignment**: Find common columns across tumor types
4. **Merge**: Combine aligned TSVs into single dataset
5. **Split by Modality**: Create separate parquet files per data type
6. **Analysis**: Analyze NaN patterns and feature quality
7. **Selective Imputation**: Impute only completely empty rows
8. **Modality-Specific Processing**: Apply tailored strategies per data type
9. **Indicator Generation**: Create missing modality flags
10. **Case Selection**: Select balanced subset with minimal missingness

## Configuration & Shared Utilities

In [ ]:
"""
Shared Configuration and Utility Functions
All paths and constants are defined here for easy modification.
"""
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import json
from collections import defaultdict
from functools import reduce

# =============================================================================
# CONFIGURATION - Modify these paths for your environment
# =============================================================================
CONFIG = {
    # Kaggle input paths
    'KAGGLE_INPUT_BASE': '/kaggle/input/organized-gbm-lgg',
    'WORKING_DIR': '/kaggle/working/quantum-classification',
    
    # Tumor types to process
    'TUMOR_TYPES': ['astrocytoma', 'glioblastoma', 'mixed_glioma', 'oligodendroglioma'],
    
    # Feature prefixes (modality types)
    'FEATURE_PREFIXES': ['Meth_', 'CNV_', 'GeneExpr_', 'miRNA_', 'SNV_', 'Prot_'],
    
    # Column names
    'ID_COL': 'case_id',
    'LABEL_COL': 'class',
    
    # Processing parameters
    'CHUNK_SIZE': 100,
    'NAN_THRESHOLD_DENSE': 90.0,
    'NAN_THRESHOLD_SNV': 95.0,
    'IMPUTE_VALUE': 0,
    'TOP_N_CASES_PER_CLASS': 78,
    
    # Output directories (relative to WORKING_DIR)
    'OUTPUTS_DIR': 'outputs',
    'OUTPUTS_COMMON_DIR': 'outputs_common',
    'FEATURE_SUBSETS_DIR': 'feature_subsets',
    'FEATURE_SUBSETS_PROCESSED_DIR': 'feature_subsets_processed_selectively',
    'FINAL_PROCESSED_DIR': 'final_processed_datasets',
    'FINAL_FILTERED_DIR': 'final_filtered_datasets',
}

# =============================================================================
# SHARED UTILITY FUNCTIONS
# =============================================================================
def safe_load_parquet(file_path, columns=None):
    """
    Safely loads a parquet file with increased thrift limits for large metadata.
    
    Args:
        file_path: Path to the parquet file
        columns: Optional list of columns to load (for memory efficiency)
    
    Returns:
        DataFrame or None if loading fails
    """
    if not os.path.exists(file_path):
        print(f"  WARNING: File not found: {file_path}")
        return None
    try:
        limit = 1 * 1024**3  # 1 GB limit
        return pd.read_parquet(
            file_path,
            columns=columns,
            thrift_string_size_limit=limit,
            thrift_container_size_limit=limit
        )
    except Exception as e:
        print(f"  ERROR loading {os.path.basename(file_path)}: {e}")
        return None


def ensure_dir(path):
    """Create directory if it doesn't exist."""
    os.makedirs(path, exist_ok=True)
    return path


def get_feature_cols(df, exclude_cols=None):
    """Get feature columns (excluding ID, label, and specified columns)."""
    exclude = {CONFIG['ID_COL'], CONFIG['LABEL_COL']}
    if exclude_cols:
        exclude.update(exclude_cols)
    return [col for col in df.columns if col not in exclude]


print("✅ Configuration and utilities loaded successfully")
print(f"   Tumor types: {CONFIG['TUMOR_TYPES']}")
print(f"   Feature prefixes: {CONFIG['FEATURE_PREFIXES']}")

## Stage 1-2: Repository Setup & Data Collection

Clone the repository and create per-tumor-type TSV files from the Kaggle datasets.

In [ ]:
# Repository setup (run on Kaggle)
# !rm -r /kaggle/working/quantum-classification
# !git clone https://github.com/vimalathithan17/quantum-classification
# %cd quantum-classification
# !git pull

# Create per-tumor-type TSV files
for tumor_type in CONFIG['TUMOR_TYPES']:
    input_dir = f"{CONFIG['KAGGLE_INPUT_BASE']}/organizedTop10_{tumor_type}"
    output_file = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_DIR']}/{tumor_type}.tsv"
    print(f"Creating TSV for {tumor_type}...")
    # Uncomment to run: !python3 py/create_multiomics.py --root {input_dir} --out {output_file} --label {tumor_type}

In [ ]:
# Analyze missing files per tumor type (diagnostic)
for tumor_type in CONFIG['TUMOR_TYPES']:
    missing_file = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_DIR']}/{tumor_type}_missing_files.tsv"
    if os.path.exists(missing_file):
        df = pd.read_csv(missing_file, delimiter='\t')
        print(f"{tumor_type}: {df.sum().sum()} missing entries")
        print("-" * 40)

## Stage 3-4: Column Alignment & Merge

Find common columns across all tumor types and align features for merging.

In [ ]:
def align_columns_across_files(file_names, input_dir, output_dir, json_log_path):
    """
    Find common columns across TSV files and save aligned versions.
    
    This solves the fundamental problem that different tumor types have 
    different feature sets - we need to find the intersection.
    """
    ensure_dir(output_dir)
    columns_by_file = {}
    
    # First pass: read headers only (memory efficient)
    print("--- Reading headers to identify column sets ---")
    for fname in file_names:
        f_path = os.path.join(input_dir, f'{fname}.tsv')
        try:
            df_header = pd.read_csv(f_path, nrows=0, delimiter='\t')
            columns_by_file[fname] = set(df_header.columns)
            print(f"  {fname}: {len(df_header.columns)} columns")
        except FileNotFoundError:
            print(f"  WARNING: {f_path} not found, skipping")
        except Exception as e:
            print(f"  ERROR reading {f_path}: {e}")

    if not columns_by_file:
        print("No files read successfully. Exiting.")
        return []

    # Find intersection
    common_columns = sorted(set.intersection(*columns_by_file.values()))
    print(f"\n✅ Found {len(common_columns)} common columns")
    
    # Log analysis
    analysis = {
        "common_columns_count": len(common_columns),
        "common_columns": common_columns,
        "uncommon_by_file": {
            fname: sorted(cols - set(common_columns))
            for fname, cols in columns_by_file.items()
        }
    }
    with open(json_log_path, 'w') as f:
        json.dump(analysis, f, indent=2)
    print(f"   Analysis saved to {json_log_path}")

    # Second pass: filter and save
    print("\n--- Saving aligned files ---")
    for fname in file_names:
        input_path = os.path.join(input_dir, f'{fname}.tsv')
        output_path = os.path.join(output_dir, f'{fname}.tsv')
        try:
            df = pd.read_csv(input_path, delimiter='\t')
            df[common_columns].to_csv(output_path, sep='\t', index=False)
            print(f"  ✅ Saved {fname}.tsv")
        except Exception as e:
            print(f"  ERROR: {fname}: {e}")
    
    return common_columns

# Run alignment
input_folder = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_DIR']}"
output_folder = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_COMMON_DIR']}"
json_file = f"{CONFIG['WORKING_DIR']}/column_analysis.json"

common_cols = align_columns_across_files(
    CONFIG['TUMOR_TYPES'], input_folder, output_folder, json_file
)
print(f"\n✅ Column alignment complete!")

In [ ]:
# Merge all aligned files into single TSV
output_common = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_COMMON_DIR']}"
merged_file = f"{output_common}/merged.tsv"

# Shell command to merge (header from first file, data from all)
tumor_files = " ".join([f"'{output_common}/{t}.tsv'" for t in CONFIG['TUMOR_TYPES']])
print(f"Merging files into {merged_file}...")
# Run: !{{ head -n1 {output_common}/{CONFIG['TUMOR_TYPES'][0]}.tsv; for f in {tumor_files}; do tail -n+2 "$f"; done; }} > {merged_file}
# !wc -l {merged_file}

In [ ]:
# Analyze feature composition by prefix
sample_file = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_DIR']}/{CONFIG['TUMOR_TYPES'][0]}.tsv"
df_header = pd.read_csv(sample_file, nrows=1, delimiter='\t')

prefix_counts = defaultdict(int)
for col in df_header.columns:
    prefix = col.split('_')[0]
    prefix_counts[prefix] += 1

print("Feature counts by modality prefix:")
for prefix, count in sorted(prefix_counts.items(), key=lambda x: -x[1]):
    print(f"  {prefix}: {count:,}")

## Stage 5: Split by Modality (Memory-Efficient Streaming)

Split the merged TSV into separate Parquet files per modality using chunked streaming to handle large files without loading everything into memory.

In [ ]:
def split_by_modality_streaming(source_file, output_dir, prefixes, chunk_size=100):
    """
    Split a large TSV into modality-specific Parquet files using chunked streaming.
    
    This is memory-efficient: processes the file in chunks rather than loading all at once.
    Uses PyArrow ParquetWriter for efficient incremental writes.
    """
    ensure_dir(output_dir)
    writers = {}
    
    print(f"Splitting {source_file} by modality...")
    chunk_iter = pd.read_csv(source_file, chunksize=chunk_size, delimiter='\t')
    
    try:
        for i, chunk in enumerate(chunk_iter):
            if i % 10 == 0:
                print(f"  Processing chunk {i + 1}...")
            
            for prefix in prefixes:
                base_cols = [CONFIG['ID_COL'], CONFIG['LABEL_COL']]
                feature_cols = [c for c in chunk.columns if c.startswith(prefix)]
                
                if not feature_cols:
                    continue
                
                subset = chunk[base_cols + feature_cols]
                table = pa.Table.from_pandas(subset, preserve_index=False)
                
                # Initialize writer on first chunk
                if i == 0 and prefix not in writers:
                    output_file = os.path.join(output_dir, f'data_{prefix}.parquet')
                    writers[prefix] = pq.ParquetWriter(output_file, table.schema)
                
                if prefix in writers:
                    writers[prefix].write_table(table)
    finally:
        # Always close writers
        for prefix, writer in writers.items():
            writer.close()
            print(f"  ✅ Closed {prefix}")
    
    print(f"\n✅ Created {len(writers)} modality files in {output_dir}")

# Run splitting
source = f"{CONFIG['WORKING_DIR']}/{CONFIG['OUTPUTS_COMMON_DIR']}/merged.tsv"
output = CONFIG['FEATURE_SUBSETS_DIR']
split_by_modality_streaming(source, output, CONFIG['FEATURE_PREFIXES'], CONFIG['CHUNK_SIZE'])

## Stage 6: Feature Quality Analysis

Analyze NaN patterns to understand data quality and guide preprocessing strategy.

In [ ]:
def analyze_feature_quality(data_dir, top_n=10):
    """
    Analyze NaN patterns in each modality file.
    
    Reports:
    - Overall NaN percentage
    - Completely empty rows (missing entire modality)
    - Sporadic missing values (within rows that have some data)
    """
    print(f"--- Feature Quality Analysis: {data_dir} ---\n")
    
    for filename in sorted(os.listdir(data_dir)):
        if not filename.endswith('.parquet'):
            continue
            
        print(f"📊 {filename}")
        df = safe_load_parquet(os.path.join(data_dir, filename))
        if df is None:
            continue
        
        rows, cols = df.shape
        feature_cols = get_feature_cols(df)
        
        # Overall NaN stats
        total_nan = df[feature_cols].isnull().sum().sum()
        total_cells = len(feature_cols) * rows
        nan_pct = (total_nan / total_cells) * 100 if total_cells > 0 else 0
        
        # Empty row analysis
        empty_mask = df[feature_cols].isnull().all(axis=1)
        n_empty = empty_mask.sum()
        
        # Sporadic NaN analysis (after removing empty rows)
        df_nonempty = df[~empty_mask]
        sporadic_nan = df_nonempty[feature_cols].isnull().sum().sum() if len(df_nonempty) > 0 else 0
        
        print(f"   Shape: {rows:,} rows × {cols:,} cols")
        print(f"   NaN: {nan_pct:.1f}% ({total_nan:,} cells)")
        print(f"   Empty rows: {n_empty} ({n_empty/rows*100:.1f}%)")
        print(f"   Sporadic NaN: {sporadic_nan:,}")
        
        # Top missing features
        if total_nan > 0:
            top_missing = df[feature_cols].isnull().sum().nlargest(top_n)
            print(f"   Top {top_n} missing features:")
            for feat, count in top_missing.items():
                print(f"      {feat[:40]}: {count}")
        print()

analyze_feature_quality(CONFIG['FEATURE_SUBSETS_DIR'])

## Stage 7: Selective Imputation & Indicator Creation

Create binary indicators for missing modalities and selectively impute only completely empty rows (preserving sporadic NaNs for models that can learn from missingness patterns).

In [ ]:
def selective_imputation(source_dir, output_dir, impute_value=0):
    """
    Create indicator columns and selectively impute only completely empty rows.
    
    Key insight: We preserve sporadic NaNs because models like LightGBM/XGBoost
    can learn from missingness patterns. Only completely empty rows (missing 
    entire modality) are filled with placeholder values.
    """
    ensure_dir(output_dir)
    print(f"--- Selective Imputation: {source_dir} → {output_dir} ---\n")
    
    for filename in sorted(os.listdir(source_dir)):
        if not filename.endswith('.parquet'):
            continue
            
        print(f"Processing {filename}...")
        df = safe_load_parquet(os.path.join(source_dir, filename))
        if df is None:
            continue
        
        feature_cols = get_feature_cols(df)
        
        # Identify completely empty rows
        empty_mask = df[feature_cols].isnull().all(axis=1)
        
        # Create indicator column
        prefix = filename.replace('data_', '').replace('.parquet', '')
        indicator_col = f"is_missing_{prefix}"
        df[indicator_col] = empty_mask.astype(int)
        
        # Impute ONLY empty rows
        df.loc[empty_mask, feature_cols] = impute_value
        
        # Save
        output_file = os.path.join(output_dir, filename)
        df.to_parquet(output_file, index=False)
        
        n_imputed = empty_mask.sum()
        print(f"  ✅ Added '{indicator_col}', imputed {n_imputed} empty rows")
    
    print(f"\n✅ Selective imputation complete")

selective_imputation(
    CONFIG['FEATURE_SUBSETS_DIR'],
    CONFIG['FEATURE_SUBSETS_PROCESSED_DIR'],
    CONFIG['IMPUTE_VALUE']
)

## Stage 8: NaN Distribution Analysis

Verify the selective imputation worked correctly by analyzing remaining NaN patterns.

In [ ]:
def analyze_nan_distribution(data_dir, bins=None):
    """
    Analyze the distribution of NaN percentages across columns.
    
    This helps verify selective imputation worked and understand
    what proportion of columns have different levels of missingness.
    """
    if bins is None:
        bins = [0, 10, 30, 50, 70, 90, 95, 97, 100]
    
    print(f"--- NaN Distribution Analysis: {data_dir} ---\n")
    
    for filename in sorted(os.listdir(data_dir)):
        if not filename.endswith('.parquet'):
            continue
            
        print(f"📊 {filename}")
        df = safe_load_parquet(os.path.join(data_dir, filename))
        if df is None:
            continue
        
        total_cols = len(df.columns)
        nan_pct = df.isnull().mean() * 100
        
        # Count columns with 0 NaN
        no_nan = (nan_pct == 0).sum()
        
        # Bin columns with NaN
        labels = [f'{bins[i]}-{bins[i+1]}%' for i in range(len(bins)-1)]
        binned = pd.cut(nan_pct[nan_pct > 0], bins=bins, labels=labels, include_lowest=True)
        dist = binned.value_counts().sort_index()
        
        print(f"   {'0%':<12}: {no_nan:>6} cols ({no_nan/total_cols*100:.1f}%)")
        for label, count in dist.items():
            pct = count / total_cols * 100
            print(f"   {label:<12}: {count:>6} cols ({pct:.1f}%)")
        print()

analyze_nan_distribution(CONFIG['FEATURE_SUBSETS_PROCESSED_DIR'])

## Stage 9: Modality-Specific Processing

Apply tailored preprocessing strategies based on each modality's characteristics:
- **Dense data** (miRNA, GeneExpr, CNV, Prot): Drop columns with >90% NaN
- **Methylation**: Preserve NaN patterns (models can learn from missingness)
- **SNV**: Handle sparse mutation data appropriately

In [ ]:
# Modality-specific processing strategies

def process_dense_data(df, nan_threshold=90.0):
    """Process dense, high-quality data by removing nearly-empty columns."""
    feature_cols = get_feature_cols(df)
    nan_pct = df[feature_cols].isnull().mean() * 100
    cols_to_drop = nan_pct[nan_pct > nan_threshold].index.tolist()
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        print(f"    Dropped {len(cols_to_drop)} columns with >{nan_threshold}% NaN")
    return df

def process_meth_data(df):
    """Preserve methylation data with NaN patterns intact."""
    print("    Preserved NaN patterns for LightGBM/XGBoost")
    return df

def process_snv_data(df):
    """Handle sparse SNV mutation data."""
    print("    Preserved sparse mutation patterns")
    return df

def apply_modality_processing(source_dir, output_dir):
    """Apply appropriate strategy to each modality file."""
    ensure_dir(output_dir)
    
    strategies = {
        'miRNA_': ('dense', process_dense_data),
        'GeneExpr_': ('dense', process_dense_data),
        'CNV_': ('dense', process_dense_data),
        'Prot_': ('dense', process_dense_data),
        'Meth_': ('preserve', process_meth_data),
        'SNV_': ('sparse', process_snv_data),
    }
    
    print(f"--- Modality-Specific Processing ---\n")
    
    for filename in sorted(os.listdir(source_dir)):
        if not filename.endswith('.parquet'):
            continue
            
        # Find matching strategy
        strategy_name, processor = None, None
        for prefix, (name, func) in strategies.items():
            if prefix in filename:
                strategy_name, processor = name, func
                break
        
        if processor is None:
            print(f"  ⚠️ No strategy for {filename}, copying as-is")
            processor = lambda df: df
            strategy_name = "copy"
        
        print(f"  📁 {filename} [{strategy_name}]")
        df = safe_load_parquet(os.path.join(source_dir, filename))
        if df is None:
            continue
        
        df = processor(df)
        df.to_parquet(os.path.join(output_dir, filename), index=False)
        print(f"    ✅ Saved ({df.shape[0]} rows, {df.shape[1]} cols)")
    
    print(f"\n✅ Modality processing complete → {output_dir}")

apply_modality_processing(
    CONFIG['FEATURE_SUBSETS_PROCESSED_DIR'],
    CONFIG['FINAL_PROCESSED_DIR']
)

## Stage 10: Indicator Summary & Case Selection

Generate a master indicator file summarizing missing modalities per case, then select the top cases per class with minimal missingness for a balanced dataset.

In [ ]:
def generate_indicator_summary(source_dir):
    """
    Create a master indicator file combining all is_missing_* columns.
    
    This enables downstream models to condition on data availability.
    """
    output_file = os.path.join(source_dir, 'indicator_features.parquet')
    print(f"--- Generating Indicator Summary ---\n")
    
    files = [f for f in os.listdir(source_dir) if f.endswith('.parquet')]
    if not files:
        print("No parquet files found")
        return None
    
    # Get base DataFrame with case_id and class
    first_file = os.path.join(source_dir, files[0])
    base_df = safe_load_parquet(first_file)
    if base_df is None:
        return None
    
    dfs_to_merge = [base_df[[CONFIG['ID_COL'], CONFIG['LABEL_COL']]]]
    
    # Extract indicator columns from each file
    for filename in files:
        df = safe_load_parquet(os.path.join(source_dir, filename))
        if df is None:
            continue
        
        indicator_cols = [c for c in df.columns if c.startswith('is_missing_')]
        if indicator_cols:
            dfs_to_merge.append(df[[CONFIG['ID_COL']] + indicator_cols])
            print(f"  Extracted {indicator_cols} from {filename}")
    
    # Merge all indicators
    if len(dfs_to_merge) > 1:
        final_df = reduce(
            lambda left, right: pd.merge(left, right, on=CONFIG['ID_COL'], how='outer'),
            dfs_to_merge
        )
        
        # Fill any missing indicators with 0
        indicator_cols = [c for c in final_df.columns if c.startswith('is_missing_')]
        final_df[indicator_cols] = final_df[indicator_cols].fillna(0).astype(int)
        
        final_df.to_parquet(output_file, index=False)
        print(f"\n✅ Saved indicator summary: {output_file}")
        print(f"   Shape: {final_df.shape}")
        print(final_df.head())
        return final_df
    
    return None

indicator_df = generate_indicator_summary(CONFIG['FINAL_PROCESSED_DIR'])

In [ ]:
def select_balanced_cases(source_dir, output_dir, indicator_file, top_n=78):
    """
    Select top N cases per class with minimal missingness for balanced training.
    
    This creates a balanced dataset where each class has equal representation
    and cases are selected based on data completeness (fewer missing modalities).
    """
    ensure_dir(output_dir)
    print(f"--- Selecting Top {top_n} Cases Per Class ---\n")
    
    # Load indicator file
    indicator_df = safe_load_parquet(indicator_file)
    if indicator_df is None:
        print("Indicator file not found")
        return
    
    # Calculate missingness score
    indicator_cols = [c for c in indicator_df.columns if c.startswith('is_missing_')]
    indicator_df['missing_score'] = indicator_df[indicator_cols].sum(axis=1)
    
    # Select top N per class (lowest missing_score)
    selected = (
        indicator_df
        .sort_values([CONFIG['LABEL_COL'], 'missing_score'], ascending=[True, True])
        .groupby(CONFIG['LABEL_COL'])
        .head(top_n)
    )
    
    keep_ids = set(selected[CONFIG['ID_COL']].unique())
    print(f"Selected {len(keep_ids)} cases total")
    print("\nCases per class:")
    print(selected[CONFIG['LABEL_COL']].value_counts())
    
    # Filter all modality files
    print("\n--- Filtering Files ---")
    files = [f for f in os.listdir(source_dir) 
             if f.endswith('.parquet') and f != 'indicator_features.parquet']
    
    for filename in files:
        df = safe_load_parquet(os.path.join(source_dir, filename))
        if df is None:
            continue
        
        filtered = df[df[CONFIG['ID_COL']].isin(keep_ids)]
        filtered.to_parquet(os.path.join(output_dir, filename), index=False)
        print(f"  ✅ {filename}: {len(filtered)} rows")
    
    # Save filtered indicators
    selected.to_parquet(os.path.join(output_dir, 'indicator_features.parquet'), index=False)
    
    print(f"\n✅ Balanced dataset saved to {output_dir}")

indicator_file = os.path.join(CONFIG['FINAL_PROCESSED_DIR'], 'indicator_features.parquet')
select_balanced_cases(
    CONFIG['FINAL_PROCESSED_DIR'],
    CONFIG['FINAL_FILTERED_DIR'],
    indicator_file,
    CONFIG['TOP_N_CASES_PER_CLASS']
)

In [ ]:
# Package final output
print("--- Packaging Output ---")
print(f"!zip -r final_dataset.zip {CONFIG['FINAL_FILTERED_DIR']}")
# Uncomment to run: !zip -r final_dataset.zip {CONFIG['FINAL_FILTERED_DIR']}